In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('./../data/cnt_events.csv')
df['dt'] = pd.to_datetime(df['dt'])
df = df.set_index('dt')
df

,event,cnt_id,cnt_type,location_name,end_location_name
dt,,,,,
2006-01-01,Прибыл_свободным,STNU5088417,40,Санкт-Петербург,NaN
2006-01-02,Прибыл_свободным,CRTU6264170,40,Санкт-Петербург,NaN
2006-01-02,Прибыл_свободным,KNYU6806102,20,Владивосток,NaN
2006-01-02,Прибыл_свободным,STNU2517447,40,Владивосток,NaN
2006-01-02,Передан_на_погрузку,STNU2517447,40,Владивосток,NaN
...,...,...,...,...,...
2007-12-31,Прибыл_груженый,TIMU5293271,20,Москва,NaN
2007-12-31,Прибыл_по_ЖД,TOWU6899546,20,Владивосток,NaN
2007-12-31,Отправлен_грузополучателю,TOWU6899546,20,Владивосток,NaN


In [5]:
df[:'2006-01-02'].loc[lambda d: d['event']=='Прибыл_свободным']

,event,cnt_id,cnt_type,location_name,end_location_name
dt,,,,,
2006-01-01,Прибыл_свободным,STNU5088417,40,Санкт-Петербург,NaN
2006-01-02,Прибыл_свободным,CRTU6264170,40,Санкт-Петербург,NaN
2006-01-02,Прибыл_свободным,KNYU6806102,20,Владивосток,NaN
2006-01-02,Прибыл_свободным,STNU2517447,40,Владивосток,NaN
2006-01-02,Прибыл_свободным,STNU3092282,40,Санкт-Петербург,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7441 entries, 2006-01-01 to 2007-12-31
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   event              7441 non-null   object
 1   cnt_id             7441 non-null   object
 2   cnt_type           7441 non-null   int64 
 3   location_name      7441 non-null   object
 4   end_location_name  1235 non-null   object
dtypes: int64(1), object(4)
memory usage: 606.8+ KB


In [ ]:
data_range = pd.date_range(start=df.index.min(), end=df.index.max())
data_range
df_empty = pd.DataFrame()

for current_day in data_range:
    df_cut = df[:current_day]
    df_last_event = df_cut.groupby([ 'cnt_id'], observed=True, as_index=False).last()
    df_count_in_location = df_last_event.loc[lambda d: (
        d['event']=='Прибыл_свободным'
        )].groupby(['location_name', 'cnt_type'], as_index=False)[['cnt_id']].count()
    df_count_in_location['dt'] = current_day
    df_empty = pd.concat([df_empty, df_count_in_location], axis=0)

df_empty = df_empty.sort_values(['dt', 'location_name', 'cnt_type'])
df_empty = df_empty.pivot(index='dt', columns=['location_name', 'cnt_type'], values='cnt_id')
df_empty = df_empty.fillna(0).astype(int)
# Создание новых названий столбцов
df_empty.columns = [f'{loc}_{cnt}' for loc, cnt in df_empty.columns]
df_empty = df_empty[[
    'Москва_20',
    'Владивосток_20',
    'Санкт-Петербург_20',
    'Москва_40',
    'Владивосток_40',
    'Санкт-Петербург_40',
]]
df_empty


,Москва_20,Владивосток_20,Санкт-Петербург_20,Москва_40,Владивосток_40,Санкт-Петербург_40
dt,,,,,,
2006-01-01,0,0,0,0,0,1
2006-01-02,0,1,0,0,0,3
2006-01-03,0,1,0,0,1,3
2006-01-04,1,1,0,1,1,3
2006-01-05,1,1,2,0,2,3
...,...,...,...,...,...,...
2007-12-27,9,5,6,5,6,7
2007-12-28,9,4,7,5,5,6
2007-12-29,9,4,7,6,6,6


In [8]:
df_empty.to_csv('./../data/cnt_empty_count.csv')